deleted service account:
https://developers.google.com/identity/protocols/OAuth2ServiceAccount#delegatingauthority

# read sheet rows

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']


In [ ]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ'
RANGE_NAME = 'opportunities_looker!A1:E'


In [ ]:
"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)



In [ ]:
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()
values = result.get('values', [])


In [ ]:
sheet_result = sheet.get(spreadsheetId=SPREADSHEET_ID).execute()

In [ ]:
sheet_result

In [ ]:
if not values:
    print('No data found.')
else:
    print(values)


# change logs for drive

In [5]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import uuid
import requests
# If modifying these scopes, delete the file token.pickle.
# SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive.file']


In [6]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'gsuite_svc_acc.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)
        

In [7]:
creds

In [8]:
# Call the Drive v3 API
results = service.files().list(
    pageSize=100, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
#         if item['name'] in ('BDR Meeting KPIs and Opportunity Targets', 'Finance Metrics'):
        print(u'{0} ({1})'.format(item['name'], item['id']))
        

Files:
BDR Meeting KPIs and Opportunity Targets (1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ)
Finance Metrics (1yRrgnEdujAsSjIDQEYIVmAHluvYVNBtw9nucoLJ7o9w)
results-20190924-142514 (1hnc47HVamnC-g0b44ouZJBZY4oQaOej23AOvAdviD_g)
Docker Cybersecurity Mtg Notes - Rolling (1FqM1NkKiJQqUwyEW5pIMqIZZxMgi6tYRGn8VxnE0BEM)
Data Culture Demystified: Strategies for Cultivating Data-Driven Decisions (17XMPkFPuSELrhhtIk2BUD9OvHijT5o6sf5uP6WRnsZQ)
DATA- Reported by me (JIRA) (1LStdLErYKgtF0RVT90NERiQ3FtVIYg7jAPigAYlxImo)
DATA- Reported by me (JIRA) (1ML1HKzR0bbfrmW6Shby56kZEVBwo1GhT4XHcuBdufuc)
DATA- Reported by me (JIRA).xlsx (0B-JSi8y-E8_fUXBWWVJvOTFpcG5aTldWUElhQktEaHdJdUFj)
Data Team Goals 2019Q3 (18kC0gA67mu4mYLU6oI7zuIIB3VQWm_xqJ1gxuhtCTqk)
Modernize Brownfield Playbook (1rBssCrTg3iOoneKuOBop9oI-jQ-Npj7Ipt-p_MU-P2A)
2019-09-12 - ProdCore - Tanzu VMW competitive updates (1YGAtO1aipIsgxv1P_eVTpQtVIsIprUhhM9u3aMKb2l8)
IMG_6581.jpg (1816n0ySPYOYc5Ww-DEBP5P_rVm3v_4A_)
Cat (108F_1EyYDUPHhcyU1cy9W5W

In [ ]:
response = service.changes().getStartPageToken().execute()

In [ ]:
response.get('startPageToken')

In [ ]:
page_token = '747'
while page_token is not None:
    response = service.changes().list(pageToken=page_token,
                                            spaces='drive').execute()
    for change in response.get('changes'):
        # Process change
        print('Change found for file: %s' % change.get('fileId'))
    if 'newStartPageToken' in response:
        # Last page, save this token for the next polling interval
        saved_start_page_token = response.get('newStartPageToken')
    page_token = response.get('nextPageToken')

In [ ]:
response

# change list with pageToken

https://developers.google.com/drive/api/v3/push

In [9]:
import json

In [25]:
channel_id = str(uuid.uuid4())
service = build('drive', 'v3', credentials=creds)
body = {
    "id": channel_id,
    "type": "web_hook",
    "address": "https://us-central1-data-team-231118.cloudfunctions.net/gsuite_meta/"
}
response = service.changes().list(pageToken = '747').execute()


In [26]:
json.dumps(response)

'{"kind": "drive#changeList", "newStartPageToken": "786", "changes": [{"kind": "drive#change", "type": "file", "changeType": "file", "time": "2019-10-02T02:53:57.322Z", "removed": false, "fileId": "108F_1EyYDUPHhcyU1cy9W5WrJbgXPPDW", "file": {"kind": "drive#file", "id": "108F_1EyYDUPHhcyU1cy9W5WrJbgXPPDW", "name": "Cat", "mimeType": "application/vnd.google-apps.folder"}}, {"kind": "drive#change", "type": "file", "changeType": "file", "time": "2019-10-02T03:53:48.002Z", "removed": false, "fileId": "18kC0gA67mu4mYLU6oI7zuIIB3VQWm_xqJ1gxuhtCTqk", "file": {"kind": "drive#file", "id": "18kC0gA67mu4mYLU6oI7zuIIB3VQWm_xqJ1gxuhtCTqk", "name": "Data Team Goals 2019Q3", "mimeType": "application/vnd.google-apps.spreadsheet"}}, {"kind": "drive#change", "type": "file", "changeType": "file", "time": "2019-10-02T06:02:08.276Z", "removed": false, "fileId": "1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ", "file": {"kind": "drive#file", "id": "1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ", "name": "BDR M

In [12]:
from google.cloud import storage
import json
from datetime import datetime

def upload_blob(bucket_name, blob_text, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client.from_service_account_json('gsuite_svc_acc.json')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_string(blob_text)


In [13]:
response
 

{'kind': 'drive#changeList',
 'newStartPageToken': '778',
 'changes': [{'kind': 'drive#change',
   'type': 'file',
   'changeType': 'file',
   'time': '2019-10-02T02:53:57.322Z',
   'removed': False,
   'fileId': '108F_1EyYDUPHhcyU1cy9W5WrJbgXPPDW',
   'file': {'kind': 'drive#file',
    'id': '108F_1EyYDUPHhcyU1cy9W5WrJbgXPPDW',
    'name': 'Cat',
    'mimeType': 'application/vnd.google-apps.folder'}},
  {'kind': 'drive#change',
   'type': 'file',
   'changeType': 'file',
   'time': '2019-10-02T03:53:48.002Z',
   'removed': False,
   'fileId': '18kC0gA67mu4mYLU6oI7zuIIB3VQWm_xqJ1gxuhtCTqk',
   'file': {'kind': 'drive#file',
    'id': '18kC0gA67mu4mYLU6oI7zuIIB3VQWm_xqJ1gxuhtCTqk',
    'name': 'Data Team Goals 2019Q3',
    'mimeType': 'application/vnd.google-apps.spreadsheet'}},
  {'kind': 'drive#change',
   'type': 'file',
   'changeType': 'file',
   'time': '2019-10-02T06:02:08.276Z',
   'removed': False,
   'fileId': '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ',
   'file': {'kind':

In [14]:
BUCKET_NAME = 'gsuite-drive-api'


In [17]:
BLOB_NAME = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
BLOB_STR = json.dumps(response)


In [18]:
upload_blob(BUCKET_NAME, BLOB_STR, BLOB_NAME)

# access token?

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client.client import OAuth2Credentials

In [ ]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(
      'gsuite_svc_acc.json', scopes=['https://www.googleapis.com/auth/drive.file'])
access_token_info = credentials.get_access_token()

# credentials = OAuth2Credentials.from_json(
#       'gsuite_credentials.json')
# credentials.get_access_token()

In [ ]:
access_token_info.access_token

In [ ]:
uuid.uuid4()

In [ ]:
import uuid
import requests
import json

In [ ]:
channel_id = str(uuid.uuid4())
SCOPES = ['https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive.readonly',
         'https://www.googleapis.com/auth/drive']

token = access_token_info.access_token

header = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

body = {
#     "kind": "api#channel",
    "id": channel_id,
    "type": "web_hook",
    "address": 'https://us-central1-data-team-231118.cloudfunctions.net/gsuite_meta/'
}

file_id = '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ'
r = requests.post(url=f'https://www.googleapis.com/drive/v3/file/{file_id}/watch?pageToken=766', 
                  data=json.dumps(body), headers=header) 


In [ ]:
r.text

In [ ]:
r_list_files = requests.get(url='https://www.googleapis.com/drive/v3/drives', headers=header)

In [ ]:
r_list_files.json()